In [ ]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

In [ ]:
(ds_train,ds_test),ds_info=tfds.load(
    'mnist',
    split=['train','test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)


In [ ]:
def preprocessing(image,label):
  return tf.cast(image,tf.float32)/255.0,label

In [ ]:
AUTOTUNE=tf.data.experimental.AUTOTUNE
BATCH_SIZE=32
ds_train=ds_train.map(preprocessing,num_parallel_calls=AUTOTUNE)
ds_train=ds_train.batch(BATCH_SIZE)
ds_train=ds_train.prefetch(AUTOTUNE)
ds_test=ds_test.map(preprocessing)
ds_test=ds_test.batch(BATCH_SIZE)
ds_test=ds_test.prefetch(AUTOTUNE)

In [ ]:
model=keras.Sequential(
    [keras.Input(shape=(28,28,1)),
     keras.layers.Conv2D(32,3,activation='relu'),
     keras.layers.MaxPooling2D(),
     keras.layers.Flatten(),
     keras.layers.Dense(10,activation='softmax')]
)


In [ ]:
model.compile(optimizer=keras.optimizers.Adam(3e-4),
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
model.fit(ds_train,validation_data=ds_test,epochs=10,verbose=2)

Epoch 1/10
1875/1875 - 11s - 6ms/step - accuracy: 0.8977 - loss: 0.3730 - val_accuracy: 0.9448 - val_loss: 0.1951
Epoch 2/10
1875/1875 - 4s - 2ms/step - accuracy: 0.9548 - loss: 0.1596 - val_accuracy: 0.9656 - val_loss: 0.1225
Epoch 3/10
1875/1875 - 4s - 2ms/step - accuracy: 0.9705 - loss: 0.1088 - val_accuracy: 0.9729 - val_loss: 0.0919
Epoch 4/10
1875/1875 - 6s - 3ms/step - accuracy: 0.9772 - loss: 0.0854 - val_accuracy: 0.9761 - val_loss: 0.0770
Epoch 5/10
1875/1875 - 10s - 5ms/step - accuracy: 0.9806 - loss: 0.0720 - val_accuracy: 0.9773 - val_loss: 0.0689
Epoch 6/10
1875/1875 - 4s - 2ms/step - accuracy: 0.9828 - loss: 0.0632 - val_accuracy: 0.9794 - val_loss: 0.0642
Epoch 7/10
1875/1875 - 4s - 2ms/step - accuracy: 0.9843 - loss: 0.0569 - val_accuracy: 0.9799 - val_loss: 0.0613
Epoch 8/10
1875/1875 - 6s - 3ms/step - accuracy: 0.9855 - loss: 0.0519 - val_accuracy: 0.9802 - val_loss: 0.0593
Epoch 9/10
1875/1875 - 4s - 2ms/step - accuracy: 0.9867 - loss: 0.0479 - val_accuracy: 0.9809 

In [ ]:
epochs=10
loss_fn=keras.losses.SparseCategoricalCrossentropy()
optimizer=keras.optimizers.Adam(3e-4)
acc_metric=keras.metrics.SparseCategoricalAccuracy()

In [ ]:
for epoch in range(epochs):
  print(f'\nTraining started for epoch {epoch}')
  for batch_idx,(x_batch,y_batch) in enumerate(ds_train):
    with tf.GradientTape() as tape:
      y_pred=model(x_batch,training=True)
      loss=loss_fn(y_batch,y_pred)
    gradients=tape.gradient(loss,model.trainable_weights)
    optimizer.apply_gradients(zip(gradients,model.trainable_weights))
    acc_metric.update_state(y_batch,y_pred)
  train_accuracy=acc_metric.result()
  print(f'Accuracy over epoch {train_accuracy}')
  acc_metric.reset_state()



Training started for epoch 0
Accuracy over epoch 0.9888333082199097

Training started for epoch 1
Accuracy over epoch 0.9898333549499512

Training started for epoch 2
Accuracy over epoch 0.9904333353042603

Training started for epoch 3
Accuracy over epoch 0.9910500049591064

Training started for epoch 4
Accuracy over epoch 0.9915333390235901

Training started for epoch 5
Accuracy over epoch 0.9922000169754028

Training started for epoch 6
Accuracy over epoch 0.9926166534423828

Training started for epoch 7
Accuracy over epoch 0.993233323097229

Training started for epoch 8
Accuracy over epoch 0.9938833117485046

Training started for epoch 9
Accuracy over epoch 0.9944166541099548


In [ ]:
#Test Loop
for batch_idx,(x_batch,y_batch) in enumerate(ds_test):
  y_pred=model(x_batch,training=False)
  acc_metric.update_state(y_batch,y_pred)
test_accuracy=acc_metric.result()
print(f'Accuracy over test set {test_accuracy}')
acc_metric.reset_state()

Accuracy over test set 0.9835000038146973
